In [1]:
#importing the libraries
import tensorflow as tf
import numpy as np
import pandas as pd
import json
import nltk
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Input, Embedding, LSTM , Dense,GlobalMaxPooling1D,Flatten
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt

2023-02-23 22:51:08.933411: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-23 22:51:15.172466: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-23 22:51:15.172490: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-23 22:51:32.116418: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [4]:
#importing the dataset
with open('../datasets/test.json') as content:
  data1 = json.load(content)
#getting all the data to lists
tags = []
inputs = []
responses={}
for intent in data1['intents']:
  responses[intent['tag']]=intent['responses']
  for lines in intent['input']:
    inputs.append(lines)
    tags.append(intent['tag'])
#converting to dataframe
data = pd.DataFrame({"inputs":inputs,
                     "tags":tags})
data

,inputs,tags
0,hello,greeting
1,hi there,greeting
2,greetings,greeting
3,nice to meet you,greeting
4,heyoo,greeting
...,...,...
77,"Hi, the following children have been placed in...",livesystem
78,Mtu anaingia vipi Kwa live system,livesystem
79,Hello can I get assistance in the opening of t...,livesystem
80,Hello can I get the technicians to go about ac...,livesystem


In [5]:
#removing punctuations
import string
data['inputs'] = data['inputs'].apply(lambda wrd:[ltrs.lower() for ltrs in wrd if ltrs not in string.punctuation])
data['inputs'] = data['inputs'].apply(lambda wrd: ''.join(wrd))
#tokenize the data
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=2000)
tokenizer.fit_on_texts(data['inputs'])
train = tokenizer.texts_to_sequences(data['inputs'])

#apply padding
from tensorflow.keras.preprocessing.sequence import pad_sequences
x_train = pad_sequences(train)

#encoding the outputs
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(data['tags'])

In [6]:
#input length
input_shape = x_train.shape[1]
print(input_shape)
#define vocabulary
vocabulary = len(tokenizer.word_index)
print("number of unique words : ",vocabulary)
#output length
output_length = le.classes_.shape[0]
print("output length: ",output_length)

34
number of unique words :  223
output length:  11


In [7]:
#creating the model
i = Input(shape=(input_shape,))
x = Embedding(vocabulary+1,10)(i)
x = LSTM(10,return_sequences=True)(x)
x = Flatten()(x)
x = Dense(output_length,activation="softmax")(x)
model  = Model(i,x)
#compiling the model
model.compile(loss="sparse_categorical_crossentropy",optimizer='adam',metrics=['accuracy'])
#training the model
train = model.fit(x_train,y_train,epochs=200)

2023-02-23 22:53:27.019646: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-02-23 22:53:27.028062: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-02-23 22:53:27.028168: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (aj): /proc/driver/nvidia/version does not exist
2023-02-23 22:53:27.105360: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/200
3/3 [==============================] - 5s 24ms/step - loss: 2.3982 - accuracy: 0.1220
Epoch 2/200
3/3 [==============================] - 0s 19ms/step - loss: 2.3855 - accuracy: 0.1951
Epoch 3/200
3/3 [==============================] - 0s 22ms/step - loss: 2.3730 - accuracy: 0.1951
Epoch 4/200
3/3 [==============================] - 0s 29ms/step - loss: 2.3556 - accuracy: 0.1829
Epoch 5/200
3/3 [==============================] - 0s 22ms/step - loss: 2.3346 - accuracy: 0.1829
Epoch 6/200
3/3 [==============================] - 0s 18ms/step - loss: 2.3078 - accuracy: 0.1951
Epoch 7/200
3/3 [==============================] - 0s 25ms/step - loss: 2.2788 - accuracy: 0.2195
Epoch 8/200
3/3 [==============================] - 0s 29ms/step - loss: 2.2461 - accuracy: 0.2439
Epoch 9/200
3/3 [==============================] - 0s 22ms/step - loss: 2.2092 - accuracy: 0.3171
Epoch 10/200
3/3 [==============================] - 0s 28ms/step - loss: 2.1795 - accuracy: 0.2195
Epoch 11/200
3/3 [=

In [8]:
#chatting
import random
while True:
  texts_p = []
  prediction_input = input('You : ')
  #removing punctuation and converting to lowercase
  prediction_input = [letters.lower() for letters in prediction_input if letters not in string.punctuation]
  prediction_input = ''.join(prediction_input)
  texts_p.append(prediction_input)
  #tokenizing and padding
  prediction_input = tokenizer.texts_to_sequences(texts_p)
  prediction_input = np.array(prediction_input).reshape(-1)
  prediction_input = pad_sequences([prediction_input],input_shape)
  #getting output from model
  output = model.predict(prediction_input)
  output = output.argmax()
  #finding the right tag and predicting
  response_tag = le.inverse_transform([output])[0]
  print("Maya : ",random.choice(responses[response_tag]))
  if response_tag == "goodbye":
    exit

1/1 [==============================] - 0s 459ms/step
Maya :  Hello there! How can I assist you today?
1/1 [==============================] - 0s 26ms/step
Maya :  Hey there, how can I assist you today?
1/1 [==============================] - 0s 22ms/step
Maya :  Hello and greetings! How can I help you today?
1/1 [==============================] - 0s 23ms/step
Maya :  The CPIMS reset password reset link is located on the login page just after the submit button or you can click on this link: https://childprotection.go.ke/accounts/password/reset/
1/1 [==============================] - 0s 24ms/step
Maya :  The CPIMS reset password reset link is located on the login page just after the submit button or you can click on this link: https://childprotection.go.ke/accounts/password/reset/
1/1 [==============================] - 0s 29ms/step
Maya :  Greetings! What brings you here today?
1/1 [==============================] - 0s 26ms/step
Maya :  Hello! I'm an AI-based virtual assistant for the CPIM

KeyboardInterrupt: Interrupted by user